In [1]:

!pip3 install nltk
!pip3 install gensim
!pip3 install numpy
!pip3 install pyLDAvis
!pip3 install spacy


  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
  Using cached click-8.1.7-py3-none-any.whl (97 kB)
  Using cached tqdm-4.66.2-py3-none-any.whl (78 kB)
     |████████████████████████████████| 291 kB 815 kB/s eta 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle 1.5.13 requires python-slugify, which is not installed.
kaggle 1.5.13 requires requests, which is not installed.
kaggle 1.5.13 requires urllib3, which is not installed.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.
     |████████████████████████████████| 24.0 MB 268 kB/s eta 0:00:01
  Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl (14.0 MB)
     |████████████████████████████████| 31.4 MB 372 kB/s eta 0:00:01    |██████▏      

In [2]:
import nltk
#nltk.download("stopwords")
import numpy as np
import json
import glob


#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#Spacy
import spacy
from nltk.corpus import stopwords

#visualization
import pyLDAvis
import pyLDAvis.gensim




In [3]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)



In [4]:
data = load_data('jsonctg-studies.json')

In [6]:
data = load_data('jsonctg-studies.json')
for item in data:
    print(item.keys())
    break

dict_keys(['protocolSection', 'resultsSection', 'documentSection', 'derivedSection', 'hasResults'])


In [8]:
for item in data:
  protocol_section = item["protocolSection"]
  document_section = item["documentSection"]
  derived_section = item["derivedSection"]
  has_results = item["hasResults"]

In [ ]:
# Assuming `keys` is a list of keys you're interested in
keys = ['protocolSection']

# Initialize an empty string to store the raw text
raw_text = ""

# Iterate over the first 100 items in data
for item in data[:100]:
  # Iterate over each key
  for key in keys:
    # Check if the key is in the item
    if key in item:
      # Print the key and the associated value
      print(f"Key: {key}, Value: {item[key]}")
      
      # Check if the value associated with the key is a string
      if isinstance(item[key], str):
        # Add the value to raw_text
        raw_text += item[key] + " "

print(raw_text)

In [6]:
# Initialize an empty string to store the raw text Brief Summary+Detailed Description
raw_text = ""

# Iterate over all items in data
for item in data:
  # Check if 'protocolSection' is in the item
  if 'protocolSection' in item:
    # Check if the value associated with 'protocolSection' is a dictionary
    if isinstance(item['protocolSection'], dict):
      # Check if 'descriptionModule' is in the 'protocolSection' dictionary
      if 'descriptionModule' in item['protocolSection']:
        # Check if the value associated with 'descriptionModule' is a dictionary
        if isinstance(item['protocolSection']['descriptionModule'], dict):
          # Check if 'briefSummary' is in the 'descriptionModule' dictionary
          if 'briefSummary' in item['protocolSection']['descriptionModule']:
            # Check if the value associated with 'briefSummary' is a string
            if isinstance(item['protocolSection']['descriptionModule']['briefSummary'], str):
              # Add the value to raw_text
              raw_text += item['protocolSection']['descriptionModule']['briefSummary'] + " "

In [7]:
stopwords = stopwords.words("english")


In [24]:
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
#THinking about how to Prepare the data for LDA model with a scientific research context

In [ ]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB',]):
    """https://spacy.io/api/annotation"""
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    texts_out = []
    for sent in texts:
        doc = nlp(sent) 
        texts_out.append(' '.join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out


sentences = raw_text.split('. ')

# Perform lemmatization
lemmatized_texts = lemmatization(sentences)

# Print the lemmatized sentences
for sentence in lemmatized_texts:
    print(sentence)

In [10]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return final


# Generate words
data_words = gen_words(lemmatized_texts)

print(data_words[0][0:20])

['study', 'design', 'expand', 'database', 'flortaucipir', 'safety', 'tau', 'bind', 'measure', 'imaging', 'provide', 'standardized', 'condition', 'flortaucipir', 'use', 'datum', 'collection', 'analysis', 'facilitate', 'companion']


In [ ]:
#topic modeling

id2word = corpora.Dictionary(data_words)
corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

In [15]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [16]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17    -0.509691  0.135074       1        1  14.416168
15     0.201494 -0.462991       2        1   8.920477
29     0.504733  0.154702       3        1   5.667239
6      0.184738  0.191669       4        1   3.908322
0      0.461471 -0.110416       5        1   3.850412
19    -0.018805 -0.322977       6        1   3.657288
12    -0.327810  0.375709       7        1   3.598887
23     0.441244  0.044163       8        1   3.461828
28    -0.424902 -0.220559       9        1   3.419557
13    -0.501466 -0.061325      10        1   3.248826
24    -0.081236  0.236560      11        1   3.111717
21    -0.119196 -0.490953      12        1   3.056477
14    -0.359702 -0.348509      13        1   2.868221
8      0.244703 -0.012484      14        1   2.723724
11     0.063251  0.499143      15        1   2.553064
5      0.433544 -0.241308      16        1   2.481619
25     0.212441 -0.223450      17        1   2.428736
9     -0.390380  0.242063      18        1   2.412591
10    -0.159545 -0.155464      19        1   2.358737
26     0.154315  0.378548      20        1   2.285031
18     0.287817  0.412110      21        1   2.222206
27    -0.095067  0.394564      22        1   2.192421
1     -0.169344  0.480023      23        1   2.118959
16     0.316885 -0.361175      24        1   2.080803
4      0.378017  0.275278      25        1   2.076081
3     -0.223174 -0.376627      26        1   2.013886
22     0.052575 -0.474266      27        1   1.984207
20    -0.352535 -0.033931      28        1   1.933048
2      0.010518 -0.023604      29        1   1.874523
7     -0.214894  0.100433      30        1   1.074945, topic_info=            Term          Freq         Total Category  logprob  loglift
23         study  55800.000000  55800.000000  Default  30.0000  30.0000
54     treatment  24706.000000  24706.000000  Default  29.0000  29.0000
71       patient  33100.000000  33100.000000  Default  28.0000  28.0000
26           use  18428.000000  18428.000000  Default  27.0000  27.0000
200  participant  16080.000000  16080.000000  Default  26.0000  26.0000
..           ...           ...           ...      ...      ...      ...
14       imaging      0.053814   1222.902826  Topic30 -12.4092  -5.4983
15       include      0.053814   5900.830289  Topic30 -12.4092  -7.0722
16        injury      0.053814   1241.553148  Topic30 -12.4092  -5.5134
17         limit      0.053814    983.754943  Topic30 -12.4092  -5.2807
19       measure      0.053814   4527.574454  Topic30 -12.4092  -6.8073

[1189 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
796      20  0.999099        ability
390      17  0.995482           able
591      28  0.998151  acceptability
594      18  0.988099     accomplish
345       6  0.998489        achieve
...     ...       ...            ...
668       5  0.999325          woman
94        5  0.999586           work
376       1  0.996314          world
749       4  0.997636          wound
226      12  0.999667           year

[580 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 16, 30, 7, 1, 20, 13, 24, 29, 14, 25, 22, 15, 9, 12, 6, 26, 10, 11, 27, 19, 28, 2, 17, 5, 4, 23, 21, 3, 8])